# Validating Your Data

## Figuring out what is in your data

In [15]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('dataset_toy/XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))

for i in range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'], 
                   [obj[0].text, obj[1].text, 
                    obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)

# creating a search object containing a list of
# duplicated rows by calling pd.DataFrame.duplicated()
search = pd.DataFrame.duplicated(df)
print(df)
print()
# displaying the list in the search list
print(search)
print()
# displaying only the row that have a boolean value of True
# which means that particular row is a duplicate
print(search[search == True])

  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True
3      3   Third    True

0    False
1    False
2    False
3     True
dtype: bool

3    True
dtype: bool


## Removing duplicates

In [17]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('dataset_toy/XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))

for i in range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'], 
                   [obj[0].text, obj[1].text, 
                    obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)

# just by calling drop_duplicates() you can
# easily remove the errant recordb
print(df.drop_duplicates())

  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True


## Creating a data map and data plan 

In [1]:
import pandas as pd
# set a maximum display.width by 55 character
pd.set_option('display.width', 55)

# creating a df that contains 3 dict items, which later
# will be grouped by the value of the first item, so
# later there will wbe only two datasets with 2 serieses
df = pd.DataFrame({'A': [0, 0, 0, 0, 0, 1, 1], 
                   'B': [1, 2, 3, 5, 4, 2, 5],
                   'C': [5, 3, 4, 1, 1, 2, 3]})

# grouping earlier df and uses dataset 'A' as the indicator
# by calling groupby() and calling describe() to obtain 
# statistics of each series in each datasets
a_group_desc = df.groupby('A').describe()
print(a_group_desc)

# for a more compact presentation you can stack the data
stacked = a_group_desc.stack()
print(stacked)

# if you just want to display a certain statistics
# using loc() lets you obtain specific column
print(a_group_desc.loc[:,(slice(None), ['count', 'mean']),])

      B                                            \
  count mean       std  min   25%  50%   75%  max   
A                                                   
0   5.0  3.0  1.581139  1.0  2.00  3.0  4.00  5.0   
1   2.0  3.5  2.121320  2.0  2.75  3.5  4.25  5.0   

      C                                            
  count mean       std  min   25%  50%   75%  max  
A                                                  
0   5.0  2.8  1.788854  1.0  1.00  3.0  4.00  5.0  
1   2.0  2.5  0.707107  2.0  2.25  2.5  2.75  3.0  
                B         C
A                          
0 count  5.000000  5.000000
  mean   3.000000  2.800000
  std    1.581139  1.788854
  min    1.000000  1.000000
  25%    2.000000  1.000000
  50%    3.000000  3.000000
  75%    4.000000  4.000000
  max    5.000000  5.000000
1 count  2.000000  2.000000
  mean   3.500000  2.500000
  std    2.121320  0.707107
  min    2.000000  2.000000
  25%    2.750000  2.250000
  50%    3.500000  2.500000
  75%    4.250000  2.75000

# Manipulating Categorical Variables 

In [2]:
import pandas as pd
print(pd.__version__)

1.0.1


## Creating categorical variable

In [26]:
import pandas as pd

# creating a categorical variable, car_colors
car_colors = pd.Series(['Blue', 'Red', 'Green'],
                        dtype='category') #specify the dtype

# list of actual car colors and using previous variable as
# the categories, and match it with the predefined acceptable
# values, when there are not match, it will output NaN
car_data = pd.Series(
    pd.Categorical(
        ['Yellow', 'Green', 'Red', 'Blue', 'Purple'],
            categories=car_colors, ordered=False))

# locating the car_data items that has NaN value
find_entries = pd.isnull(car_data)

print(car_colors)
print()
print(car_data)
print()
print(find_entries[find_entries == True])


0     Blue
1      Red
2    Green
dtype: category
Categories (3, object): [Blue, Green, Red]

0      NaN
1    Green
2      Red
3     Blue
4      NaN
dtype: category
Categories (3, object): [Blue, Green, Red]

0    True
4    True
dtype: bool


## Renaming Levels 

In [69]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'],
                        dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Blue', 'Red'],
            categories=car_colors, ordered=False))

# using cat.categories() to set a new value
car_colors.cat.categories = ['Purple', 'Yellow', 'Mauve']
car_data.cat.categories = car_colors

print(car_data)

0    Purple
1    Yellow
2     Mauve
3    Purple
4     Mauve
dtype: category
Categories (3, object): [Purple, Yellow, Mauve]


## Combining levels 

In [70]:
car_colors = pd.Series(['Blue', 'Red', 'Green'],
                        dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Green', 'Red', 'Green'],
            categories=car_colors, ordered=False))

# Adding Blue_Red category to car_data
car_data = car_data.cat.set_categories(
    ['Blue', 'Red', 'Green', 'Blue_Red'])
# locating 'Red' in car_data and printing it
print (car_data.loc[car_data.isin(['Red'])])
# locating 'Red' and change its value to 'Red_Blue'
car_data.loc[car_data.isin(['Red'])] = 'Blue_Red'
# locating 'Blue' and change its value to 'Red_Blue'
car_data.loc[car_data.isin(['Blue'])] = 'Blue_Red'

# setting the cateogories just to "Green" and "Blue_Red"
car_data = car_data.cat.set_categories(
    ["Green", "Blue_Red"])

print()
print(car_data)

2    Red
4    Red
dtype: category
Categories (4, object): [Blue, Red, Green, Blue_Red]

0    Blue_Red
1       Green
2    Blue_Red
3       Green
4    Blue_Red
5       Green
dtype: category
Categories (2, object): [Green, Blue_Red]


# Dealing with Date in Your Data

## Formatting date and time values 

In [75]:
import datetime as dt

# Assigning current time to variable now
now = dt.datetime.now()

# displaying variable now as a string
print(str(now))
# transforming variable now into a string with a specific defining
# %a : the abrreviation of the day
# %d : date
# %B : Month
# %Y : Year
print(now.strftime('%a, %d %B %Y'))

2020-06-14 15:54:29.379967
Sun, 14 June 2020


## Using the right time transformation 

In [93]:
import datetime as dt

now = dt.datetime.now()
# Transforming one time into another time with timedelta()
# you can use : days, seconds, minutes, hours, weeks
timevalue = now + dt.timedelta(hours=2)

# %H : Hour
# %M : Minute
# %S : Second
print(now.strftime('%H:%M:%S'))
print(timevalue.strftime('%H:%M:%S'))
print(timevalue - now)

15:59:16
17:59:16
2:00:00


# Dealing with Missing Data


## Finding the missing data 

In [136]:
import pandas as pd
import numpy as np

# build a df with two missing data
# represented by np.Nan (using numpy)
# and None (using python)
s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print(s)
print()
# Displaying all the data and assign 
# False value to the non-missing data
# and True to the missing data
print(s.isnull())

print()
# Showing only the data that have the missing data
print(s[s.isnull()])

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
5    6.0
6    NaN
dtype: float64
0    False
1    False
2    False
3     True
4    False
5    False
6     True
dtype: bool

3   NaN
6   NaN
dtype: float64


## Encoding missingness

### filling in missing data or dropping it


In [137]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print(s)
print()
# filling the missing data with the mean of the data
print(s.fillna(int(s.mean())))
print()
# dropping the missing data
print(s.dropna())

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
5    6.0
6    NaN
dtype: float64

0    1.0
1    2.0
2    3.0
3    3.0
4    5.0
5    6.0
6    3.0
dtype: float64

0    1.0
1    2.0
2    3.0
4    5.0
5    6.0
dtype: float64


## Imputing missing data

In [161]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# list with some missing values
s = [[1, 2, 3, np.NaN, 5, 6, None]]

# creating a imputer to replace the missing value
# with stratigey parameter to define how to replace the missing data
# you can use mean, median and most_frequent as the arguments
imp = SimpleImputer(missing_values=np.nan,
             strategy='mean')

# provide the statistics for the imputer to use by calling fit()
imp.fit([[1, 2, 3, 4, 5, 6, 7]])

# calling transfomr in s to fill the missing data
# and start with index 0 by using tolist()
x = pd.Series(imp.transform(s).tolist()[0])

print(x)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
6    7.0
dtype: float64


# Slicing and Dicing: Filtering and Selecting Data

## Slicing rows 

In [170]:
# making a 3-D array
x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],], 
              [[21,22,23], [24,25,26], [27,28,29]]])

print(x)

# slicing row 1 from 3-D array x
x[1]

[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[11 12 13]
  [14 15 16]
  [17 18 19]]

 [[21 22 23]
  [24 25 26]
  [27 28 29]]]


array([[11, 12, 13],
       [14, 15, 16],
       [17, 18, 19]])

## Slicing columns 

In [172]:
x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],], 
              [[21,22,23], [24,25,26], [27,28,29]]])
print(x)
# slicing the entire rows using (:)
# in the column 1
x[:,1]

[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[11 12 13]
  [14 15 16]
  [17 18 19]]

 [[21 22 23]
  [24 25 26]
  [27 28 29]]]


array([[ 4,  5,  6],
       [14, 15, 16],
       [24, 25, 26]])

## Dicing 

In [251]:
x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],], 
              [[21,22,23], [24,25,26], [27,28,29]]])
print(x)
print()
# dicing the row 2 from column 1
print(x[2,1])
# dicing the item 1 from every row in column 0
print(x[:,0,1])
# dicing item 2 in row 1 from every column
print(x[1,:,2])
print()
# dicing rows 1 and 2 in columns 1 and 2
print(x[1:3, 1:3])

[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[11 12 13]
  [14 15 16]
  [17 18 19]]

 [[21 22 23]
  [24 25 26]
  [27 28 29]]]

[24 25 26]
[ 2 12 22]
[13 16 19]

[[[14 15 16]
  [17 18 19]]

 [[24 25 26]
  [27 28 29]]]


# Concatenating and Transforming 

## Adding new cases and variables 

In [4]:
import pandas as pd

# creating a df
df = pd.DataFrame({'A': [2, 3, 1], 
                   'B': [1, 2, 3],
                   'C': [5, 3, 4]})

# the df which later will be appended into the other one
df1 = pd.DataFrame({'A': [4], 
                   'B': [4],
                   'C': [4]})
print(df)
print()
# appending df1 into df ( just adding a new row)
df = df.append(df1)
print(df)
print()
# resetting the index of df
# the drop parameter funtions (default: False) 
# to drop the previous index (after appending)
df = df.reset_index(drop=True)
print(df)

# adding a new row/case in df using loc()
df.loc[df.last_valid_index() + 1] = [5, 5, 5]
print()
print(df)

# making a df that consist of a new column
df2 = pd.DataFrame({'D' : [1, 2, 3, 4, 5]})

# joining df and df2 (adding a new column/variable in df)
df = pd.DataFrame.join(df, df2)
print()
print(df)

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
0  4  4  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4
4  5  5  5

   A  B  C  D
0  2  1  5  1
1  3  2  3  2
2  1  3  4  3
3  4  4  4  4
4  5  5  5  5


## Removing Data 

In [43]:
import pandas as pd

df = pd.DataFrame({'A': [2, 3, 1], 
                   'B': [1, 2, 3],
                   'C': [5, 3, 4]})

# dropping a row/case from a df
df = df.drop(df.index[[1]])
print(df)

# Dropping a column by 'B' as the name of the column
# and 1 as the axis of the column (usually always 1)
df = df.drop('A', 1)
print()
print(df)

   A  B  C
0  2  1  5
2  1  3  4

   B  C
0  1  5
2  3  4


## Sorting and Shuffling 

In [97]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': [2, 1, 2, 3, 3, 5, 4], 
                   'B': [1, 2, 3, 5, 4, 2, 5],
                   'C': [5, 3, 4, 1, 1, 2, 3]})

# sorting the data by 'A' column first and if there are same numbers
# it will refer to column 'B' next
df = df.sort_values(by=['A', 'B'], ascending=[True, True])
df = df.reset_index(drop=True)
print(df)

# make a list of all index there are in df
index = df.index.tolist()
# shuffling the items in the index list
np.random.shuffle(index)
# locating the value in a certain index according
# to the new shuffled index and putting it in the df
df = df.loc[df.index[index]]
df = df.reset_index(drop=True)
print()
print(df)

   A  B  C
0  1  2  3
1  2  1  5
2  2  3  4
3  3  4  1
4  3  5  1
5  4  5  3
6  5  2  2

   A  B  C
0  1  2  3
1  3  4  1
2  4  5  3
3  3  5  1
4  5  2  2
5  2  1  5
6  2  3  4


# Aggregating Data at Any Level 

In [99]:
import pandas as pd

df = pd.DataFrame({'Map': [0, 0, 0, 1, 1, 2, 2],
                   'Values' : [1, 2, 3, 5, 4, 2, 5]})

# making some new variables/columns which consist of 
# the counting of statistical value from the items in 
# Values column that corresponds to the grouping of items
# in 'Map' 
df['S'] = df.groupby('Map')['Values'].transform(np.sum)
df['M'] = df.groupby('Map')['Values'].transform(np.mean)
df['V'] = df.groupby('Map')['Values'].transform(np.var)

print(df)

   Map  Values  S    M    V
0    0       1  6  2.0  1.0
1    0       2  6  2.0  1.0
2    0       3  6  2.0  1.0
3    1       5  9  4.5  0.5
4    1       4  9  4.5  0.5
5    2       2  7  3.5  4.5
6    2       5  7  3.5  4.5
